# Visualize Sinusoidal Positional Encoding

### Basic static plot of all dimensions:

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

# Parameters
max_len = 90
d_model = 32

pos = np.arange(max_len)
data = []

for i in range(d_model // 2):  # d_model is even
    div_term = 10000 ** (2 * i / d_model)
    sin_vals = np.sin(pos / div_term)
    cos_vals = np.cos(pos / div_term)

    for p, s, c in zip(pos, sin_vals, cos_vals):
        data.append({"pos": p, "value": s, "dim": f"sin({i})"})
        data.append({"pos": p, "value": c, "dim": f"cos({i})"})

df = pd.DataFrame(data)

# Plot all dimensions
fig = px.line(
    df, x="pos", y="value", color="dim",
    title="Sinusoidal Positional Encoding (all dimensions)"
)
fig.show()


In [4]:
import numpy as np
import plotly.graph_objects as go

# Parameters
max_len = 90
d_model = 32
pos = np.arange(max_len)

# Prepare traces for each dimension
all_traces = []
for i in range(d_model // 2):
    div_term = 10000 ** (2 * i / d_model)
    sin_vals = np.sin(pos / div_term)
    cos_vals = np.cos(pos / div_term)
    all_traces.append({"sin": sin_vals, "cos": cos_vals, "dim": i})

# Initial plot with first dimension
fig = go.Figure()
fig.add_trace(go.Scatter(x=pos, y=all_traces[0]["sin"], mode='lines', name='sin(0)'))
fig.add_trace(go.Scatter(x=pos, y=all_traces[0]["cos"], mode='lines', name='cos(0)'))

# Create slider steps
steps = []
for i, trace in enumerate(all_traces):
    step = dict(
        method="update",
        args=[{
                  "y": [trace["sin"], trace["cos"]],
                  "name": [f"sin({i})", f"cos({i})"]
              }],
        label=f"{i}"
    )
    steps.append(step)

sliders = [dict(active=0, currentvalue={"prefix": "Dimension: "}, pad={"t": 50}, steps=steps)]
fig.update_layout(
    sliders=sliders, title="Sinusoidal PE - Interactive Dimension",
    xaxis_title="Position", yaxis_title="Value", width=800, height=400
)
fig.show()


In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.colors as pc

# -----------------------------
# Parameters: default values
# -----------------------------
max_len = 90  # default maximum sequence length
d_model = 32  # default embedding dimension

# You can change these values to explore
custom_max_len = 90
custom_d_model = 32

# -----------------------------
# Generate sinusoidal PE data
# -----------------------------
pos = np.arange(custom_max_len)
dims = np.arange(custom_d_model)
data = []

for i in dims:
    div_term = 10000 ** (2 * (i // 2) / custom_d_model)  # i//2 to match sin/cos pair
    vals = np.sin(pos / div_term) if i % 2 == 0 else np.cos(pos / div_term)
    for p, v in zip(pos, vals):
        data.append({"position": p, "dimension": i, "value": v})

df = pd.DataFrame(data)

# ------------------------------------------
# 1️⃣ First figure: slider on dimension
# ------------------------------------------
# English comments: dimension slider updates which sin/cos dimension to show
fig1 = px.line(
    df[df['dimension'] == 0], x="position", y="value",
    color="dimension", line_group="dimension",
    title=f"Sinusoidal PE - Dimension Slider (max_len={custom_max_len}, d_model={custom_d_model})",
    color_discrete_sequence=pc.qualitative.Dark24
)

# Create dimension slider manually
steps = []
for i in dims:
    step = dict(
        method="update",
        args=[{
                  "y": [df[df['dimension'] == i]['value']],
                  "x": [df[df['dimension'] == i]['position']],
                  "line.color": [pc.qualitative.Dark24[i % len(pc.qualitative.Dark24)]]
              }],
        label=str(i)
    )
    steps.append(step)

fig1.update_layout(
    sliders=[dict(active=0, steps=steps, currentvalue={"prefix": "Dimension: "})]
)

fig1.show()

# ------------------------------------------
# 2️⃣ Second figure: slider on position
# ------------------------------------------
# Prepare data in pivot form for position slider
pivot_df = df.pivot(index='dimension', columns='position', values='value').reset_index()

# Initial trace: first position
pos0 = pos[0]
fig2 = px.line(
    pivot_df, x='dimension', y=pos0,
    title=f"Sinusoidal PE - Position Slider (max_len={custom_max_len}, d_model={custom_d_model})",
    color_discrete_sequence=pc.qualitative.Dark24
)

# Position slider steps
steps_pos = []
for p in pos:
    step = dict(
        method="update",
        args=[{"y": [pivot_df[p]]}],
        label=str(p)
    )
    steps_pos.append(step)

fig2.update_layout(
    sliders=[dict(active=0, steps=steps_pos, currentvalue={"prefix": "Position: "})],
    xaxis_title="Dimension",
    yaxis_title="Value"
)

fig2.show()

# ------------------------------------------
# 3️⃣ Third figure: 3D surface plot
# ------------------------------------------
fig3 = px.scatter_3d(
    df, x='position', y='dimension', z='value',
    color='value', color_continuous_scale=pc.sequential.Viridis,
    title=f"Sinusoidal PE 3D Surface (max_len={custom_max_len}, d_model={custom_d_model})",
    hover_data={'position': True, 'dimension': True, 'value': True}
)

fig3.update_traces(marker=dict(size=4))
fig3.show()


In [7]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.colors as pc

# -----------------------------
# Parameters
# -----------------------------
custom_max_len = 90
custom_d_model = 32
N = 3  # number of consecutive lines to show
pos = np.arange(custom_max_len)
dims = np.arange(custom_d_model)
data = []

# -----------------------------
# Generate sinusoidal PE data
# -----------------------------
for i in dims:
    div_term = 10000 ** (2 * (i // 2) / custom_d_model)
    vals = np.sin(pos / div_term) if i % 2 == 0 else np.cos(pos / div_term)
    for p, v in zip(pos, vals):
        data.append({"position": p, "dimension": i, "value": v})

df = pd.DataFrame(data)

# -----------------------------
# 1️⃣ Dimension slider (show N consecutive dimensions)
# -----------------------------
start_dim = 0
dims_to_show = np.arange(start_dim, min(start_dim + N, custom_d_model))
df_subset = df[df['dimension'].isin(dims_to_show)]

fig1 = px.line(
    df_subset,
    x='position',
    y='value',
    color='dimension',
    title=f"Sinusoidal PE - Dimension slider (N={N})",
    color_discrete_sequence=pc.qualitative.Dark24
)
fig1.show()

# -----------------------------
# 2️⃣ Position slider (show N consecutive positions)
# -----------------------------
# pivot to wide-form: rows=dimension, columns=position
pivot_df = df.pivot(index='dimension', columns='position', values='value').reset_index()

start_pos = 0
positions_to_show = np.arange(start_pos, min(start_pos + N, custom_max_len))
positions_to_show = [int(p) for p in positions_to_show if p in pivot_df.columns]

fig2 = px.line(
    pivot_df,
    x='dimension',
    y=positions_to_show,  # list of column names
    title=f"Sinusoidal PE - Position slider (N={N})",
    color_discrete_sequence=pc.qualitative.Dark24
)
fig2.show()


### Gradio Interface:

In [9]:
import gradio as gr
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.colors as pc


# -----------------------------
# Function to generate sinusoidal PE DataFrame
# -----------------------------
def generate_pe(max_len, d_model):
    pos = np.arange(max_len)
    dims = np.arange(d_model)
    data = []
    for i in dims:
        div_term = 10000 ** (2 * (i // 2) / d_model)
        vals = np.sin(pos / div_term) if i % 2 == 0 else np.cos(pos / div_term)
        for p, v in zip(pos, vals):
            data.append({"position": p, "dimension": i, "value": v})
    return pd.DataFrame(data)


df_global = generate_pe(90, 32)  # default data


# -----------------------------
# Dimension slider plot
# -----------------------------
def plot_dimension(start_dim, N, max_len=90, d_model=32):
    df = generate_pe(max_len, d_model)
    dims_to_show = np.arange(start_dim, min(start_dim + N, d_model))
    df_subset = df[df['dimension'].isin(dims_to_show)]
    fig = px.line(
        df_subset, x='position', y='value', color='dimension',
        color_discrete_sequence=pc.qualitative.Dark24,
        title=f"Sinusoidal PE - Dimension slider (N={N})"
    )
    return fig


# -----------------------------
# Position slider plot
# -----------------------------
def plot_position(start_pos, N, max_len=90, d_model=32):
    df = generate_pe(max_len, d_model)
    pivot_df = df.pivot(index='dimension', columns='position', values='value').reset_index()
    positions_to_show = np.arange(start_pos, min(start_pos + N, max_len))
    positions_to_show = [int(p) for p in positions_to_show if p in pivot_df.columns]
    fig = px.line(
        pivot_df, x='dimension', y=positions_to_show,
        color_discrete_sequence=pc.qualitative.Dark24,
        title=f"Sinusoidal PE - Position slider (N={N})"
    )
    return fig


# -----------------------------
# Gradio Interface
# -----------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Sinusoidal Positional Encoding Explorer (Gradio)")

    with gr.Tab("Dimension Slider"):
        dim_slider = gr.Slider(0, 31, value=0, step=1, label="Start Dimension")
        N_input_dim = gr.Number(value=3, label="Number of consecutive dimensions N")
        max_len_input = gr.Number(value=90, label="Max sequence length")
        d_model_input = gr.Number(value=32, label="d_model")
        dim_plot = gr.Plot()
        dim_slider.change(plot_dimension, [dim_slider, N_input_dim, max_len_input, d_model_input], dim_plot)
        N_input_dim.change(plot_dimension, [dim_slider, N_input_dim, max_len_input, d_model_input], dim_plot)

    with gr.Tab("Position Slider"):
        pos_slider = gr.Slider(0, 89, value=0, step=1, label="Start Position")
        N_input_pos = gr.Number(value=3, label="Number of consecutive positions N")
        max_len_input2 = gr.Number(value=90, label="Max sequence length")
        d_model_input2 = gr.Number(value=32, label="d_model")
        pos_plot = gr.Plot()
        pos_slider.change(plot_position, [pos_slider, N_input_pos, max_len_input2, d_model_input2], pos_plot)
        N_input_pos.change(plot_position, [pos_slider, N_input_pos, max_len_input2, d_model_input2], pos_plot)

demo.launch()


/mnt/832acd65-7396-480e-aa76-dca6765861b0/research-repo/projects/2025.09_hydrology_Switzerland/codes/swiss-river-network-benchmark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
